In [1]:
## Corridors Crash Data

### 1. Needs number of crashes, fatalities, injuries, severity categorization, Mode for each corridor in study area
### 2. Database: DOTDEVRHPGSQL01
### 3. Tables: study_area, nysdot_all, Districts_Congres

In [1]:
from ris import db2  #library designed for SQL database connection and querying
from IPython.display import clear_output
import datetime 
from datetime import date
import pandas as pd
import os


clear_output()
timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')
print 'Notebook run: {}'.format(timestamp)
print os.getcwd()
# %load_ext sql 

Notebook run: 2019-08-06 16:09
E:\RIS\Staff Folders\Samuel\Requests\SH\Crash_Data


In [2]:
#Database connections
#gdb = db2.PostgresDb('dotdevpgsql02', 'GISGRID', quiet = True)
cdb = db2.PostgresDb('DOTDEVRHPGSQL01', 'CRASHDATA', quiet = True)

User name (CRASHDATA):soge
Password (CRASHDATA)········


In [3]:
df = pd.read_excel('Corridors List_20190725.xlsx', sheet_name='Crash Database  ')
#df

In [4]:
cols=df.columns[8:18]

In [5]:
new_headers=map(str, cols)
#new_headers

In [6]:
new_headers.insert(0,str(df.columns[1]))

In [7]:
corridors_crash_data = (db2.query_to_table(cdb, """
                        select id_b,
                        count(nys.case_num) tot_crashes,
                        sum(f.inj) tot_fat,
                        sum(nys.num_of_inj) tot_inj, 
                        sum(length(nys.ext_of_inj::text) - length(replace(nys.ext_of_inj::text, 'A'::text, ''::text))) AS A,
                        sum(length(nys.ext_of_inj::text) - length(replace(nys.ext_of_inj::text, 'B'::text, ''::text))) AS B,
                        sum(length(nys.ext_of_inj::text) - length(replace(nys.ext_of_inj::text, 'C'::text, ''::text))) AS C,                       
                        ABS(sum(length(TRIM(nys.ext_of_inj::text))-(nys.num_of_fat + nys.num_of_inj))) UNK,
                        sum(case when nys.accd_type_int= 1 then num_of_inj else 0 end) as Ped,
                        sum(case when nys.accd_type_int= 2 then num_of_inj else 0 end) as Bike,
                        sum(length(nys.ext_of_inj::text)) AS KSI
                        FROM working.study_area sa
                        left join nysdot_all nys
                        on sa.segmentid = nys.segmentid
                        left join fatality_nycdot_current f
                        on sa.segmentid = f.segmentid
                        where nys.case_yr>2004 and nys.case_yr<2015
                        group by id_b""" 
                        ))

In [8]:
headers=map(str, corridors_crash_data.columns)
#headers

In [9]:
dictionary = [{i:j} for i,j in zip(headers, new_headers)]
for i in dictionary:        
    corridors_crash_data=corridors_crash_data.rename(columns=i)

#corridors_crash_data.columns
#corridors_crash_data

In [10]:
#pd.merge(df.loc[:,df.columns[:8]],corridors_crash_data, how='left').shape
data = df.loc[:, df.columns[:8]].merge(corridors_crash_data, how='left')

In [11]:
data.to_csv('data.csv')